# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [5]:
from transformers import AutoTokenizer #, AutoModelForSequenceClassification

In [6]:
from util import util_elastic

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [7]:
import importlib
# importlib.reload(util_elastic)

In [8]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 40)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 200)


In [9]:
PATH_DOC =  '../data/juris_tcu_index/doc.csv'
PATH_QUERY =  '../data/juris_tcu_index/query.csv'
PATH_QREL =  '../data/juris_tcu_index/qrel.csv'

In [10]:
%%time
import logging
logging.getLogger("haystack").setLevel(logging.WARNING) #WARNING, INFO

CPU times: user 214 µs, sys: 0 ns, total: 214 µs
Wall time: 216 µs


# Data load

In [11]:
df_doc = pd.read_csv(PATH_DOC)

In [12]:
df_query = pd.read_csv(PATH_QUERY)

In [13]:
df_qrel = pd.read_csv(PATH_QREL)

In [14]:
df_doc.shape, df_query.shape, df_qrel.shape

((13255, 28), (16045, 11), (94809, 3))

In [15]:
df_query.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,NaN,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE
1,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",NaN,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE
2,21087,"A contratação de serviços por preços superiores às referências legais (Sinapi e Sicro) deve ser,...",NaN,NaN,Licitação,932,PRESTAÇÃO DE CONTAS,Acórdão 3936/2013 - Segunda Câmara,2013-07-09,RELATOR,ANA ARRAES
3,35016,"Não se aplica, excepcionalmente, multa aos gestores por grave infração à norma legal se a irregu...",NaN,NaN,Responsabilidade,775,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ
4,29370,"Em contratatações de TI, não cabe aceitar propostas de preço com incidência de encargos majorado...",NaN,NaN,Licitação,932,REPRESENTAÇÃO,Acórdão 3231/2011 - Plenário,2011-12-07,RELATOR,AROLDO CEDRAZ


### Quantitative calculation

Tokenization loading

In [16]:
nome_modelo_ranking_pt = 'unicamp-dl/mMiniLM-L6-v2-pt-v2'
nome_caminho_modelo_pt = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_ranking_pt
assert os.path.exists(nome_caminho_modelo_pt), f"Path para {nome_caminho_modelo_pt} não existe!"

nome_modelo_monot5_3b = 'unicamp-dl/mt5-3B-mmarco-en-pt'
# "A mono-ptT5 reranker model (850 mb) pretrained in the BrWac corpus, finetuned for 100k steps on Portuguese translated version of MS MARCO passage dataset. The portuguese dataset was translated using Google Translate.")

nome_caminho_modelo_3b = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_monot5_3b
assert os.path.exists(nome_caminho_modelo_3b), f"Path para {nome_caminho_modelo_3b} não existe!"

tokenizador_pt_minilm = AutoTokenizer.from_pretrained(nome_caminho_modelo_pt)
tokenizador_pt_monot5_3b = AutoTokenizer.from_pretrained(nome_caminho_modelo_3b)

/home/borela/miniconda3/envs/relevar-busca/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


In [17]:
def retorna_num_tokens(parm_texto:str, parm_tokenizador:AutoTokenizer):
    return len(parm_tokenizador.tokenize(parm_texto))


In [18]:
df_query['LEN_TEXT_CHAR'] = df_query['TEXT'].apply(len)
df_query['LEN_TEXT_CHAR_LOG'] = round(np.log(df_query['TEXT'].apply(len))).astype(int)
df_query['NUM_WORD'] = df_query['TEXT'].apply(lambda x: len(x.split()))
df_query['NUM_TOKENS_MONOT5_3B'] = df_query['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_monot5_3b)
df_query['NUM_TOKENS_MINILM'] = df_query['TEXT'].apply(retorna_num_tokens, parm_tokenizador=tokenizador_pt_minilm)

In [19]:
df_query.columns, df_qrel.columns, df_doc.columns, 

(Index(['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE',
        'NORMATIVE_AUTHOR_NAME', 'LEN_TEXT_CHAR', 'LEN_TEXT_CHAR_LOG', 'NUM_WORD', 'NUM_TOKENS_MONOT5_3B', 'NUM_TOKENS_MINILM'],
       dtype='object'),
 Index(['QUERY_ID', 'DOC_ID', 'TYPE'], dtype='object'),
 Index(['TEXT', 'ID', 'NAME', 'DATE_REFERENCE', 'CLASS', 'HAS_SCOPE_NOTE', 'HAS_DEFINITION', 'HAS_SYNONYM', 'HAS_EXAMPLE', 'HAS_ENGLISH_TRANSLATION', 'HAS_SPECIALIZATION', 'HAS_RELATED_TERM',
        'HAS_SPANISH_TRANSLATION', 'HAS_GENERALIZATION', 'TEXT_DEFINITION', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'TEXT_SCOPE_NOTE', 'TEXT_EXAMPLE', 'TEXT_ENGLISH_TRANSLATION',
        'TEXT_SPANISH_TRANSLATION', 'TEXT_SPECIALIZATION', 'TEXT_GENERALIZATION', 'COUNT_INDEX_AREA', 'COUNT_INDEX_THEME', 'COUNT_INDEX_SUBTHEME', 'COUNT_INDEX_EXTRA', 'COUNT_INDEX_TOTAL'],
       dtype='object'))

In [20]:
# Verificar as colunas com valores NaN
columns_with_nan = df_query.columns[df_query.isna().any()].tolist()

print(columns_with_nan)

['REFERENCE_LIST', 'PARADIGMATIC', 'NORMATIVE_PROCESS_TYPE']


In [21]:
df_query = df_query.fillna("")

In [26]:
df_doc.head()

,TEXT,ID,NAME,DATE_REFERENCE,CLASS,HAS_SCOPE_NOTE,HAS_DEFINITION,HAS_SYNONYM,HAS_EXAMPLE,HAS_ENGLISH_TRANSLATION,HAS_SPECIALIZATION,HAS_RELATED_TERM,HAS_SPANISH_TRANSLATION,HAS_GENERALIZATION,TEXT_DEFINITION,TEXT_SYNONYM,TEXT_RELATED_TERM,TEXT_SCOPE_NOTE,TEXT_EXAMPLE,TEXT_ENGLISH_TRANSLATION,TEXT_SPANISH_TRANSLATION,TEXT_SPECIALIZATION,TEXT_GENERALIZATION,COUNT_INDEX_AREA,COUNT_INDEX_THEME,COUNT_INDEX_SUBTHEME,COUNT_INDEX_EXTRA,COUNT_INDEX_TOTAL
0,"O termo é ""Abandono de cargo"".\nAbandono de cargo tem definição: ""Configura abandono de cargo a ...",1,Abandono de cargo,20230614,Termo,1,1,1,0,1,0,1,1,0,Configura abandono de cargo a ausência intencional do servidor ao serviço por mais de trinta dia...,Abandono de função e Abandono do cargo.,"Abandono de emprego, Deserção, Falta grave, Função pública, Demissão de servidor, Inassiduidade ...",Artigo 323 Código Penal,NaN,Abandonment of office.,Abandono de puesto.,NaN,NaN,0,1,0,0,1
1,"O termo é ""Abastecimento de tropas e meios"".\nAbastecimento de tropas e meios é uma especializaç...",2,Abastecimento de tropas e meios,20230614,Termo,0,0,0,0,1,0,0,1,1,NaN,NaN,NaN,NaN,NaN,Supply of troops and means.,Abastecimiento de tropas y medios e Suministro de tropas y médios.,Abastecimento.,NaN,0,0,0,1,1
2,"O termo é ""Tropa"".\nTropa tem definição: ""Termo coletivo que designa o pessoal de uma organizaçã...",3,Tropa,20230614,Termo,0,1,0,0,1,0,1,1,0,Termo coletivo que designa o pessoal de uma organização militar.,NaN,Organização militar e Missão.,NaN,NaN,Troop.,Tropa.,NaN,NaN,0,0,0,0,0
3,"O termo é ""Abono de faltas"".\nAbono de faltas tem definição: ""É a relevação da falta, de forma q...",4,Abono de faltas,20230614,Termo,1,1,1,0,0,0,1,0,0,"É a relevação da falta, de forma que o servidor não sofra qualquer desconto pelo dia não trabalh...","Faltas, Abono das faltas e Abono de ausências.",Inassiduidade habitual e Afastamento de pessoal.,Usado para servidores regidos pela CLT (art. 473).,NaN,NaN,NaN,NaN,NaN,0,0,0,1,1
4,"O termo é ""Abono de permanência em serviço"".\nAbono de permanência em serviço tem definição: ""O ...",5,Abono de permanência em serviço,20230614,Termo,0,1,1,0,0,0,1,0,0,O abono de permanência será pago ao servidor que tiver preenchido as exigências para aposentador...,"Abono de permanência, Adicional de permanência e Abono permanência.","Aposentadoria voluntária, Contribuição previdenciária e Regime Próprio de Previdência Social.",NaN,NaN,NaN,NaN,NaN,NaN,0,7,0,4,11


In [30]:
# Verificar as colunas com valores NaN
columns_with_nan = df_doc.columns[df_doc.isna().any()].tolist()

print(columns_with_nan)

['TEXT_DEFINITION', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'TEXT_SCOPE_NOTE', 'TEXT_EXAMPLE', 'TEXT_ENGLISH_TRANSLATION', 'TEXT_SPANISH_TRANSLATION', 'TEXT_SPECIALIZATION', 'TEXT_GENERALIZATION']


In [31]:
df_doc = df_doc.fillna("")

In [33]:
# Converter as colunas para o tipo string
df_doc[columns_with_nan] = df_doc[columns_with_nan].astype(str)

Concatenate terms (by qrel) and synonyms (by doc)

In [69]:
df = df_qrel.merge(df_doc[['ID', 'NAME', 'TEXT_SYNONYM', 'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM']], how='inner', left_on='DOC_ID', right_on='ID')

In [70]:
df.shape

(94809, 9)

In [76]:
del df['ID']

In [71]:
df.head()

,QUERY_ID,DOC_ID,TYPE,ID,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM
0,158,15147,INDEXACAO_EXTRA,15147,Referência,Parâmetro e Fonte de pesquisa.,Fonte de informação.,1,1
1,37,15147,INDEXACAO_EXTRA,15147,Referência,Parâmetro e Fonte de pesquisa.,Fonte de informação.,1,1
2,178,15147,INDEXACAO_EXTRA,15147,Referência,Parâmetro e Fonte de pesquisa.,Fonte de informação.,1,1
3,14564,15147,INDEXACAO_EXTRA,15147,Referência,Parâmetro e Fonte de pesquisa.,Fonte de informação.,1,1
4,9219,15147,INDEXACAO_EXTRA,15147,Referência,Parâmetro e Fonte de pesquisa.,Fonte de informação.,1,1


In [72]:
df['NAME_SYNONYM'] = df.apply(lambda row: row['NAME'] if row['TEXT_SYNONYM'] == '' else row['NAME'] + ' - ' + row['TEXT_SYNONYM'], axis=1)


In [73]:
df['NAME_RELATED_TERM'] = df.apply(lambda row: row['NAME'] if row['TEXT_RELATED_TERM'] == '' else row['NAME'] + ' -- ' + row['TEXT_RELATED_TERM'], axis=1)


In [74]:
df['NAME_SYNONYM_RELATED_TERM'] = df.apply(lambda row: row['NAME_SYNONYM'] if row['TEXT_RELATED_TERM'] == '' else row['NAME_SYNONYM'] + ' -- ' + row['TEXT_RELATED_TERM'], axis=1)


In [75]:
df[(df['HAS_SYNONYM']==1) & (df['HAS_RELATED_TERM']==0)].head(10)

,QUERY_ID,DOC_ID,TYPE,ID,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM,NAME_SYNONYM,NAME_RELATED_TERM,NAME_SYNONYM_RELATED_TERM
3650,199,1391,INDEXACAO_EXTRA,1391,Requisito,Requisitos.,,1,0,Requisito - Requisitos.,Requisito,Requisito - Requisitos.
3651,197,1391,SUBTEMA,1391,Requisito,Requisitos.,,1,0,Requisito - Requisitos.,Requisito,Requisito - Requisitos.
3652,174,1391,SUBTEMA,1391,Requisito,Requisitos.,,1,0,Requisito - Requisitos.,Requisito,Requisito - Requisitos.
3653,100,1391,INDEXACAO_EXTRA,1391,Requisito,Requisitos.,,1,0,Requisito - Requisitos.,Requisito,Requisito - Requisitos.
3654,89,1391,SUBTEMA,1391,Requisito,Requisitos.,,1,0,Requisito - Requisitos.,Requisito,Requisito - Requisitos.
3655,244,1391,SUBTEMA,1391,Requisito,Requisitos.,,1,0,Requisito - Requisitos.,Requisito,Requisito - Requisitos.
3656,13626,1391,INDEXACAO_EXTRA,1391,Requisito,Requisitos.,,1,0,Requisito - Requisitos.,Requisito,Requisito - Requisitos.
3657,14470,1391,INDEXACAO_EXTRA,1391,Requisito,Requisitos.,,1,0,Requisito - Requisitos.,Requisito,Requisito - Requisitos.
3658,51947,1391,INDEXACAO_EXTRA,1391,Requisito,Requisitos.,,1,0,Requisito - Requisitos.,Requisito,Requisito - Requisitos.
3659,14494,1391,INDEXACAO_EXTRA,1391,Requisito,Requisitos.,,1,0,Requisito - Requisitos.,Requisito,Requisito - Requisitos.


In [77]:
df = df_query.merge(df, how='inner', left_on='ID', right_on='QUERY_ID')

In [84]:
df.columns

Index(['ID', 'TEXT', 'REFERENCE_LIST', 'PARADIGMATIC', 'AREA_NAME', 'AREA_ID_DESCRIPTOR', 'NORMATIVE_PROCESS_TYPE', 'NORMATIVE_IDENTIFICATION', 'NORMATIVE_DATE', 'NORMATIVE_AUTHOR_TYPE',
       'NORMATIVE_AUTHOR_NAME', 'LEN_TEXT_CHAR', 'LEN_TEXT_CHAR_LOG', 'NUM_WORD', 'NUM_TOKENS_MONOT5_3B', 'NUM_TOKENS_MINILM', 'QUERY_ID', 'DOC_ID', 'TYPE', 'NAME', 'TEXT_SYNONYM',
       'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM', 'NAME_SYNONYM', 'NAME_RELATED_TERM', 'NAME_SYNONYM_RELATED_TERM'],
      dtype='object')

In [78]:
df.head()

,ID,TEXT,REFERENCE_LIST,PARADIGMATIC,AREA_NAME,AREA_ID_DESCRIPTOR,NORMATIVE_PROCESS_TYPE,NORMATIVE_IDENTIFICATION,NORMATIVE_DATE,NORMATIVE_AUTHOR_TYPE,NORMATIVE_AUTHOR_NAME,LEN_TEXT_CHAR,LEN_TEXT_CHAR_LOG,NUM_WORD,NUM_TOKENS_MONOT5_3B,NUM_TOKENS_MINILM,QUERY_ID,DOC_ID,TYPE,NAME,TEXT_SYNONYM,TEXT_RELATED_TERM,HAS_SYNONYM,HAS_RELATED_TERM,NAME_SYNONYM,NAME_RELATED_TERM,NAME_SYNONYM_RELATED_TERM
0,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE,321,6,50,96,71,13568,6665,INDEXACAO_EXTRA,Tempo ficto,,Tempo de serviço.,0,1,Tempo ficto,Tempo ficto -- Tempo de serviço.,Tempo ficto -- Tempo de serviço.
1,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE,321,6,50,96,71,13568,94,TEMA,Aposentadoria especial,,"Regime Geral de Previdência Social, Salário-de-benefício e Perfil profissiográfico previdenciário.",0,1,Aposentadoria especial,"Aposentadoria especial -- Regime Geral de Previdência Social, Salário-de-benefício e Perfil prof...","Aposentadoria especial -- Regime Geral de Previdência Social, Salário-de-benefício e Perfil prof..."
2,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE,321,6,50,96,71,13568,15525,SUBTEMA,Policial,,,0,0,Policial,Policial,Policial
3,13568,"É ilegal a contagem de tempo ficto de serviço prestado sob a égide da Lei 3.313/1957, proporcion...",Lei Complementar 51/1985 || Lei Ordinária 3313/1957,,Pessoal,1131,APOSENTADORIA,Acórdão 168/2014 - Segunda Câmara,2014-01-28,RELATOR,JOSÉ JORGE,321,6,50,96,71,13568,1131,AREA,Pessoal,,"Sisac, Despesa com pessoal e HCAAF.",0,1,Pessoal,"Pessoal -- Sisac, Despesa com pessoal e HCAAF.","Pessoal -- Sisac, Despesa com pessoal e HCAAF."
4,11614,"SÚMULA TCU 283: Para fim de habilitação, a Administração Pública não deve exigir dos licitantes ...",,SUMULA,Licitação,932,ADMINISTRATIVO,Acórdão 1613/2013 - Plenário,2013-06-26,RELATOR,JOSÉ JORGE,191,5,30,54,45,11614,14782,SUBTEMA,Documentação,Guarda da documentação.,,1,0,Documentação - Guarda da documentação.,Documentação,Documentação - Guarda da documentação.


In [79]:
df.shape

(94809, 27)

In [128]:
%%time
data_index = {'term' : [],
            'synonym': [],
            'related_term': [],
            'synonym_related_term': [],}
# Ordenar o DataFrame por 'QUERY_ID' em ordem ascendente
df_sorted = df.sort_values('QUERY_ID')
# Iterar sobre os grupos
for cnt, (query_id, group) in tqdm(enumerate(df_sorted.groupby('QUERY_ID'))):
    # print(f"Query ID: {query_id}")
    # print(group)
    group_synonym = ''
    group_term = ''
    group_related_term = ''
    group_synonym_related_term = ''

    # Iterar sobre os registros do grupo
    for cnt, (_, row_doc) in enumerate(group.iterrows()):
        if cnt==0: # salva copia dos dados da query
            row_query = row_doc.copy()

        doc_id = row_doc['DOC_ID']
        # Faça o que precisa com cada registro (doc_id) para o query_id atual
        
        # Exemplo: Imprimir o doc_id
        # print(f"Doc ID: {doc_id}")

        group_term = group_term + ', ' + row_doc['NAME'] if group_term != '' else row_doc['NAME']
        if row_doc['HAS_SYNONYM']==1:
            group_synonym = group_synonym + ' ' + row_doc['NAME_SYNONYM'] if group_synonym != '' else row_doc['NAME_SYNONYM']
        if row_doc['HAS_RELATED_TERM']==1:
            group_related_term = group_related_term + ' ' + row_doc['NAME_RELATED_TERM'] if group_related_term != '' else row_doc['NAME_RELATED_TERM']
        if (row_doc['HAS_RELATED_TERM']==1) or (row_doc['HAS_SYNONYM']==1):
            group_synonym_related_term = group_synonym_related_term + ' ' + row_doc['NAME_SYNONYM_RELATED_TERM'] if group_synonym_related_term != '' else row_doc['NAME_SYNONYM_RELATED_TERM']
    # Adicione aqui qualquer lógica adicional que você precise executar para cada query_id

    row_json_term= {
        'id': row_query['ID'],
        'content': row_query['TEXT'],
        'meta': {},
    }               
    for column_name in df.columns:
        if column_name not in ['TEXT', 'QUERY_ID', 'DOC_ID', 'TYPE', 'NAME', 'TEXT_SYNONYM',
                               'TEXT_RELATED_TERM', 'HAS_SYNONYM', 'HAS_RELATED_TERM', 'NAME_SYNONYM',
                               'NAME_RELATED_TERM', 'NAME_SYNONYM_RELATED_TERM']:  # columns of doc
            row_json_term['meta'][column_name.lower()] = row_query[column_name]        
    
    row_json_synonym = row_json_term.copy()
    row_json_related_term = row_json_term.copy()
    row_json_synonym_related_term = row_json_term.copy()

    row_json_term['content'] += '\n' + group_term + '.'
    if len(group_synonym) > 0:
        row_json_synonym['content'] += '\n' + group_synonym
    if len(group_related_term) > 0:
        row_json_related_term['content'] += '\n' + group_related_term
    if len(group_synonym_related_term) > 0:
        row_json_synonym_related_term['content'] += '\n' + group_synonym_related_term


    data_index['term'].append(row_json_term)
    data_index['synonym'].append(row_json_synonym)
    data_index['related_term'].append(row_json_related_term)
    data_index['synonym_related_term'].append(row_json_synonym_related_term)


16045it [00:07, 2217.00it/s]

CPU times: user 7.25 s, sys: 60.3 ms, total: 7.31 s
Wall time: 7.27 s


In [129]:
len(data_index['term'])

16045

In [130]:
list_type_index = ['term', 'synonym', 'related_term', 'synonym_related_term' ]

In [132]:
ind_test=1
for index_type in list_type_index:
    print(index_type, data_index[index_type][ind_test]['content'], '\n')


term SÚMULA TCU 2: Configura-se como vencimento, para efeito da concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, o valor do símbolo correspondente ao cargo em comissão exercido pelo funcionário, à época do seu falecimento.
Cargo em comissão, Doença, Pessoal, Viúvo, Pensão especial. 

synonym SÚMULA TCU 2: Configura-se como vencimento, para efeito da concessão da pensão especial com fundamento na Lei nº 3.738, de 04/04/60, o valor do símbolo correspondente ao cargo em comissão exercido pelo funcionário, à época do seu falecimento.
Cargo em comissão - Ocupante de cargo em comissão, Cargo de direção, chefia e assessoramento, Cargo comissionado, Cargo de confiança, Exercente de cargo em comissão e Cargo de direção, chefia ou assessoramento. Doença - Enfermidade e Moléstia. Viúvo - Viúva, Cônjuge sobrevivente, Cônjuge supérstite e Consorte supérstite. Pensão especial - Pensão especial da Lei 3738/60, Pensão especial da Lei 6782/80, Pensão da lei 3.738/60 e Pensão es

# Index creation

In [125]:
index_dict = util_elastic.return_indexes('indir', parm_print=True)

Index: indir_juris_tcu
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu', 'uuid': 'RD_aAWppQH6vjNxKUdL_0w', 'pri': '1', 'rep': '1', 'docs.count': '16045', 'docs.deleted': '16045', 'store.size': '334.9mb', 'pri.store.size': '334.9mb'}

Index: indir_juris_tcu_index
{'health': 'yellow', 'status': 'open', 'index': 'indir_juris_tcu_index', 'uuid': 'fGjzBrQzRUWTZ6KlnbfLqw', 'pri': '1', 'rep': '1', 'docs.count': '13255', 'docs.deleted': '2755', 'store.size': '281.7mb', 'pri.store.size': '281.7mb'}



In [134]:
INDEX_NAME = 'indir_juris_tcu'

In [26]:
# util_elastic.delete_index('indir_juris_tcu')

In [136]:
%%time
dict_index = {}
for index_type in list_type_index:
    print(f"\n********Creating index {INDEX_NAME+'_'+index_type}")
    dict_index[index_type] = util_elastic.create_index(parm_index_name=INDEX_NAME+'_'+index_type,
                                                       parm_data_carga_json=data_index[index_type],
                                                       parm_embedding_dim=1024)

Creating index indir_juris_tcu_term

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>
Creating index indir_juris_tcu_synonym

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>
Creating index indir_juris_tcu_related_term

before write

Qtd de documentos 0

Qtd de embeddings 0

Documento.id= 5: None

after write

Qtd de documentos 16045

Qtd de embeddings 0

Documento.id= 5: <Document: id=5, content='SÚMULA TCU 1: Não se compreendem como vencimento, para efeito de concessão da pensão especial com fu...'>
Creating index in

In [140]:
nome_modelo_embedding_model_sts_rufimelo = "rufimelo/Legal-BERTimbau-sts-large-ma-v3"
nome_caminho_modelo = "/home/borela/fontes/relevar-busca/modelo/" + nome_modelo_embedding_model_sts_rufimelo
assert os.path.exists(nome_caminho_modelo), f"Path para {nome_caminho_modelo} não existe!"


In [143]:
%%time
for index_type in list_type_index:
    print(f"\n********Updating embeddings of index {INDEX_NAME+'_'+index_type}")
    util_elastic.update_index_embedding_sts(parm_index=dict_index[index_type], parm_path_model=nome_caminho_modelo)


********Updating embeddings of index indir_juris_tcu_term


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_synonym


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_related_term


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]


********Updating embeddings of index indir_juris_tcu_synonym_related_term


Updating embeddings:   0%|          | 0/16045 [00:00<?, ? Docs/s]

CPU times: user 10min 41s, sys: 3.99 s, total: 10min 45s
Wall time: 13min 49s
